In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from huggingface_hub import login


In [ ]:
json_file = '/Updated_XBRL.json'
df = pd.read_json(json_file)

In [ ]:
# Display the first few rows of the dataframe
print(df.head())

     id                                              query  \
0  3525  What is the US GAAP XBRL tag for Accounts Paya...   
1  3537  What is the US GAAP XBRL tag for Accounts Rece...   
2  4958  What is the US GAAP XBRL tag for Total Assets ...   
3  1378  What is the US GAAP XBRL tag for Cost of Goods...   
4  3525  What is the US GAAP XBRL tag for Total Revenue...   

                                                text         answer  
0  file:c-826.xml\n<{http://fasb.org/us-gaap/2023...  Answer:f-2543  
1  file:c-10.xml\n<{http://fasb.org/us-gaap/2023}...  Answer:f-2366  
2  file:c-17.xml\n<{http://fasb.org/us-gaap/2023}...  Answer:f-3645  
3  file:c-4.xml\n<{http://xbrl.sec.gov/dei/2023}S...    Answer:f-78  
4  file:c-33.xml\n<{http://xbrl.sec.gov/dei/2023}...   Answer:f-138  


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B", token= '')
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B",token='')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 134.81 MiB is free. Process 84193 has 39.42 GiB memory in use. Of the allocated memory 38.76 GiB is allocated by PyTorch, and 182.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# 0-shot, answering questions every time in a size of 5
def generate_responses(questions, batch_size=5):
    responses = []
    tokenizer.pad_token = tokenizer.eos_token
    for i in range(0, len(questions), batch_size):
        batch_questions = questions[i:i + batch_size]
        inputs = tokenizer.batch_encode_plus(
            batch_questions,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to('cuda')

        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length= 750,  # Set to the maximum length you need
            pad_token_id=tokenizer.eos_token_id,  # Set the end token
            no_repeat_ngram_size=2  # Optional: Prevent the generation of repeated 2-length n-grams
        )

        batch_responses = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        responses.extend(batch_responses)

    return responses

In [ ]:
df['evidence_text'] = generate_responses(df['query'].tolist())

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [ ]:
# Save the updated dataframe to a new CSV file
output_json_file = 'Llama-3.1-MOF.json'
df.to_json(output_json_file, index=False)

In [ ]:
print(f"Responses have been generated and saved to {output_json_file}")

Responses have been generated and saved to Llama-3.1-MOF.csv


In [ ]:
# Load and display the first few rows of the output CSV file for verification
output_df = pd.read_json(output_json_file, orient='records', lines=True)
print(output_df.head())

  Question_type                                           Question  \
0           ACM                                       BSD-1-Clause   
1           ACM                                            AFL-3.0   
2           ACM                                            APL-1.0   
3            OS  Is the license OSI approved? Creative Commons ...   
4            OS             Is the license OSI approved? Unlicense   

   Generated Answer                        answer  \
0               NaN          1-clause BSD License   
1               NaN  Academic Free License v. 3.0   
2               NaN   Adaptive Public License 1.0   
3               NaN                            No   
4               NaN                           Yes   

                                       evidence_text  
0  BSD-1-Clause\nCopyright (c) 2013-2015, Open So...  
1  AFL-3.0 Aims to Simplify the Licensing Process...  
2  APL-1.0APLD-2.01\nAPLC-3.1APLB-4.02\nB-5.12B-B...  
3  Is the license OSI approved? Creative